In [5]:
# Function to find leaders in the code
def find_leaders(code):
    leaders = [0]  # The first instruction is always a leader
    i = 0
    while i < len(code):
        if code[i] == '\n':
            i += 1
            continue
        if i > 0 and code[i - 1] == '\n':
            leaders.append(i)
        i += 1
    return leaders

# Function to find basic blocks in the code
def find_basic_blocks(code, leaders):
    basic_blocks = []
    i = 0
    while i < len(leaders) - 1:
        start = leaders[i]
        end = leaders[i + 1]
        block = code[start:end]
        basic_blocks.append(block)
        i += 1
    return basic_blocks

# Function to create the program flow graph from basic blocks
def create_pfg(basic_blocks):
    pfg = {}
    i = 0
    while i < len(basic_blocks):
        block = basic_blocks[i]
        pfg[i] = []
        if 'goto' in block:
            goto_index = block.index('goto')
            goto_target = block[goto_index + 1]
            target_block_index = int(goto_target.split('_')[-1])
            pfg[i].append(target_block_index)
        elif 'return' not in block:
            pfg[i].append(i + 1)
        i += 1
    return pfg

# Function to find the natural loops in the program flow graph
def find_natural_loops(pfg):
    loops = []
    for node, children in pfg.items():
        for child in children:
            if child <= node:
                continue
            visited = set()
            visited.add(node)
            stack = [(node, child)]
            while stack:
                start, end = stack.pop()
                visited.add(end)
                for neighbor in pfg[end]:
                    if neighbor == start:
                        loops.append((start, end))
                    elif neighbor not in visited:
                        stack.append((start, neighbor))
    return loops


In [6]:
# Sample code
code = """
a = 0
b = 1
if a < b:
    goto _1
else:
    goto _2
_1:
c = 2
goto _3
_2:
c = 3
goto _3
_3:
d = 4
e = 5
if d < e:
    goto _4
else:
    goto _5
_4:
f = 6
goto _6
_5:
f = 7
goto _6
_6:
return
"""

# Find leaders
leaders = find_leaders(code)

# Find basic blocks
basic_blocks = find_basic_blocks(code, leaders)

# Create program flow graph
pfg = create_pfg(basic_blocks)

# Find natural loops
loops = find_natural_loops(pfg)

# Print the loops
print(loops)


ValueError: invalid literal for int() with base 10: 'o'

In [7]:
# Define a class for basic blocks
class BasicBlock:
    def __init__(self):
        self.instructions = []  # List to store instructions
        self.successors = []    # List to store successors
        self.leader = False     # Flag to indicate if the block is a leader
        self.loop = False       # Flag to indicate if the block is part of a loop

# Define a function to detect loops
def detect_loops(program):
    # Step 1: Finding leaders
    leaders = []        # List to store leaders
    for i in range(len(program)):
        if i == 0 or program[i-1] == "goto" or program[i-1] == "if":
            leaders.append(i)

    # Step 2: Creating basic blocks
    basic_blocks = []   # List to store basic blocks
    for i in range(len(leaders)):
        block = BasicBlock()
        if i < len(leaders) - 1:
            block.instructions = program[leaders[i]:leaders[i+1]]
        else:
            block.instructions = program[leaders[i]:]
        basic_blocks.append(block)

    # Step 3: Building program flow graph
    for i in range(len(basic_blocks)):
        last_instruction = basic_blocks[i].instructions[-1]
        if last_instruction == "goto":
            basic_blocks[i].successors.append(basic_blocks[leaders.index(i+1)])
        elif last_instruction == "if":
            basic_blocks[i].successors.append(basic_blocks[leaders.index(i+1)])
            basic_blocks[i].successors.append(basic_blocks[leaders.index(i+2)])
        else:
            if i < len(basic_blocks) - 1:
                basic_blocks[i].successors.append(basic_blocks[i+1])

    # Step 4: Finding natural loops
    for i in range(len(basic_blocks)):
        for successor in basic_blocks[i].successors:
            if leaders.index(successor) < leaders.index(i):
                basic_blocks[i].loop = True

    # Step 5: Printing basic blocks and loop information
    for i in range(len(basic_blocks)):
        print("Basic Block", i)
        print("Instructions:", basic_blocks[i].instructions)
        print("Successors:", [leaders.index(block) for block in basic_blocks[i].successors])
        print("Is Leader:", basic_blocks[i].leader)
        print("Is Loop:", basic_blocks[i].loop)
        print("")

# Example usage
program = ["a = 1", "b = 2", "if a < b", "    goto", "else", "    goto", "c = a + b", "d = c * 2"]
detect_loops(program)


Basic Block 0
Instructions: ['a = 1', 'b = 2', 'if a < b', '    goto', 'else', '    goto', 'c = a + b', 'd = c * 2']
Successors: []
Is Leader: False
Is Loop: False



In [28]:
# Function to find leaders in the code
def find_leaders(code):
   leaders = [0] # The first instruction is always a leader.

    for i in range(len(code)):
        # Check if the current instruction is a branch or a label.
        if is_branch(code[i]) or get_label_name(code[i]) in labels:
            # If so, the next instruction is a leader.
            leaders.append(i + 1)

    return leaders

# Function to find basic blocks
def find_basic_blocks(code, leaders):
    blocks = []
    for i in range(len(leaders)):
        if i == len(leaders) - 1:
            blocks.append(code[leaders[i]:])
        else:
            blocks.append(code[leaders[i]:leaders[i+1]])
    return blocks

# Function to build the program flow graph
def build_flow_graph(blocks):
    graph = {}
    for i in range(len(blocks)):
        graph[i] = []
        for j in range(i+1, len(blocks)):
            if 'goto ' + str(j) in blocks[i]:
                graph[i].append(j)
                break
            elif 'if' in blocks[i]:
                graph[i].append(j)
                if 'goto ' + str(j) in blocks[i+1]:
                    break
    return graph

# Function to find natural loops in the program flow graph
def find_natural_loops(graph):
    loops = []
    for i in range(len(graph)):
        for j in graph[i]:
            for k in graph[j]:
                if i in graph[k]:
                    loops.append((i, j, k))
    return loops

# Sample code to test the functions
code = '''
count = 0
if ( count > 20 ) GOTO 8 
    count = count + 1
    increment = 2 * count
    result = result + increment
GOTO 3
END
'''
leaders = find_leaders(code)
blocks = find_basic_blocks(code, leaders)
graph = build_flow_graph(blocks)
loops = find_natural_loops(graph)

# Print the results
print(code)
print('Leaders:', leaders)
print('Basic Blocks:', blocks)
print('Program Flow Graph:', graph)
print('Natural Loops:', loops)



count = 0
if ( count > 20 ) GOTO 8 
    count = count + 1
    increment = 2 * count
    result = result + increment
GOTO 3
END


count = 0
if ( count > 20 ) GOTO 8 
    count = count + 1
    increment = 2 * count
    result = result + increment
GOTO 3
END

Leaders: [0]
Basic Blocks: ['\ncount = 0\nif ( count > 20 ) GOTO 8 \n    count = count + 1\n    increment = 2 * count\n    result = result + increment\nGOTO 3\nEND\n']
Program Flow Graph: {0: []}
Natural Loops: []
